In [53]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
dfTrue = pd.read_csv('./data/True.csv')
dfFake= pd.read_csv('./data/Fake.csv')

In [7]:
dfTrue['isFake'] = 0
dfFake['isFake'] = 1

In [22]:
combinedDF = pd.concat([dfTrue, dfFake], axis=0)
combinedAndShuffeldDF = combinedDF.sample(frac=1).reset_index(drop=True)

In [23]:
df = combinedAndShuffeldDF

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)

In [37]:
df['text'] = df['text'].apply(preprocess_text)
df['title'] = df['title'].apply(preprocess_text)

In [54]:
vectorizer_text = TfidfVectorizer(max_features=5000)
vectorizer_title = TfidfVectorizer(max_features=5000)

vectorized_text = vectorizer_text.fit_transform(df['text'])
vectorized_title = vectorizer_title.fit_transform(df['title'])

X = hstack([vectorized_text, vectorized_title])

y = df['isFake']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)


In [47]:
models = {
    'Gradient Boosting': GradientBoostingClassifier(),
    'Ada Boosting': AdaBoostClassifier()
}

In [58]:
for model_name, model in models.items():
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    print(f"{model_name} Accuracy: %{accuracy*100}")

Gradient Boosting Accuracy: %99.6881959910913
Ada Boosting Accuracy: %99.83964365256125
